# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [18]:
# Your code here
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from itertools import combinations

# Load the dataset
df = pd.read_csv("ToothGrowth.csv")

# Display first few rows
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [19]:
# Your code here
# Ensure categorical variables are correctly formatted
df["supp"] = df["supp"].astype("category")  
df["dose"] = df["dose"].astype("category")  

# Fit an ANOVA model
model = smf.ols('len ~ C(supp) + C(dose) + C(supp):C(dose)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Display the ANOVA table
print(anova_table)

                      sum_sq    df          F        PR(>F)
C(supp)           205.350000   1.0  15.571979  2.311828e-04
C(dose)          2426.434333   2.0  91.999965  4.046291e-18
C(supp):C(dose)   108.319000   2.0   4.106991  2.186027e-02
Residual          712.106000  54.0        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [20]:
# Your comment here
""" 
If both supp and dose are significant, both factors influence tooth growth.
If only dose is significant, dosage has a greater impact than supplement type.
If the interaction is significant, the effectiveness of a supplement depends on the dosage level.If not, dose influences growth independently of supplement type.
"""

' \nIf both supp and dose are significant, both factors influence tooth growth.\nIf only dose is significant, dosage has a greater impact than supplement type.\nIf the interaction is significant, the effectiveness of a supplement depends on the dosage level.If not, dose influences growth independently of supplement type.\n'

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [21]:
# Your code here
#Split data by supplement type
oj_group = df[df["supp"] == "OJ"]["len"]
vc_group = df[df["supp"] == "VC"]["len"]

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(oj_group, vc_group, equal_var=False)  # Welch's t-test

# Display results
print(f"T-statistic: {t_stat:.4f}, P-value: {p_value:.4f}")


T-statistic: 1.9153, P-value: 0.0606


Now run a t-test between these two groups and print the associated two-sided p-value: 

In [22]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
# Split data into two groups based on supplement type
oj_group = df[df["supp"] == "OJ"]["len"]
vc_group = df[df["supp"] == "VC"]["len"]

# Perform an independent two-sided t-test
t_stat, p_value = stats.ttest_ind(oj_group, vc_group, equal_var=False)  # Welch's t-test

# Print the p-value
print(f"Two-sided p-value: {p_value:.4f}")

Two-sided p-value: 0.0606


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [23]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
# Fit an ANOVA model with only supplement type
model = smf.ols('len ~ C(supp)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Print the ANOVA table
print(anova_table)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [27]:
for group in df.groupby(['supp', 'dose'], observed=True)['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', np.float64(0.5))
('OJ', np.float64(1.0))
('OJ', np.float64(2.0))
('VC', np.float64(0.5))
('VC', np.float64(1.0))
('VC', np.float64(2.0))


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [26]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
# Group data by supplement and dose
grouped_data = {group: data.values for group, data in df.groupby(['supp', 'dose'], observed=False)['len']}

# Generate all pairwise combinations
group_combinations = list(combinations(grouped_data.keys(), 2))

# Perform t-tests for each combination
for (group1, group2) in group_combinations:
    data1, data2 = grouped_data[group1], grouped_data[group2]
    t_stat, p_value = stats.ttest_ind(data1, data2, equal_var=False)  # Welch's t-test
    
    # Print results
    print(f"Comparison: {group1} vs {group2} | p-value: {p_value:.4f}")

Comparison: ('OJ', np.float64(0.5)) vs ('OJ', np.float64(1.0)) | p-value: 0.0001
Comparison: ('OJ', np.float64(0.5)) vs ('OJ', np.float64(2.0)) | p-value: 0.0000
Comparison: ('OJ', np.float64(0.5)) vs ('VC', np.float64(0.5)) | p-value: 0.0064
Comparison: ('OJ', np.float64(0.5)) vs ('VC', np.float64(1.0)) | p-value: 0.0460
Comparison: ('OJ', np.float64(0.5)) vs ('VC', np.float64(2.0)) | p-value: 0.0000
Comparison: ('OJ', np.float64(1.0)) vs ('OJ', np.float64(2.0)) | p-value: 0.0392
Comparison: ('OJ', np.float64(1.0)) vs ('VC', np.float64(0.5)) | p-value: 0.0000
Comparison: ('OJ', np.float64(1.0)) vs ('VC', np.float64(1.0)) | p-value: 0.0010
Comparison: ('OJ', np.float64(1.0)) vs ('VC', np.float64(2.0)) | p-value: 0.0965
Comparison: ('OJ', np.float64(2.0)) vs ('VC', np.float64(0.5)) | p-value: 0.0000
Comparison: ('OJ', np.float64(2.0)) vs ('VC', np.float64(1.0)) | p-value: 0.0000
Comparison: ('OJ', np.float64(2.0)) vs ('VC', np.float64(2.0)) | p-value: 0.9639
Comparison: ('VC', np.float6

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.